In [3]:
import pandas as pd
import tweepy
import csv

consumer_key = 'JVG6jTdGES3dQzLwjTti6cxI3'
consumer_secret = 'LTfxYvaBv3pG7T7pNEt22gfGXktDAsyuJAiusdthbkEvPgUXmI'
access_token = '1048577269013381121-B2QSHS2FaTNeB4c9oVY02Z4Qs448Md'
access_token_secret = 'x3xEezyvlR7dnI6AtYjJjeVj9MU45ccJ3cOJtmMVZ7ZcB'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

tweet_url = pd.read_csv("OLAprilMay19.txt", index_col= None,
header = None, names = ["links"])

af = lambda x: x["links"].split("/")[-1]
tweet_url['id'] = tweet_url.apply(af, axis=1)
tweet_url.head()

ids = tweet_url['id'].tolist()
total_count = len(ids)
chunks = (total_count - 1) // 50 + 1

def fetch_tw(ids):
    list_of_tw_status = api.statuses_lookup(ids, tweet_mode= "extended")
    empty_data = pd.DataFrame()
    for status in list_of_tw_status:
            tweet_elem = {"date": status.created_at,
                     "tweet_id":status.id,
                     "tweet":status.full_text,
                     "User location":status.user.location,
                     "Retweet count":status.retweet_count,
                     "Like count":status.favorite_count,
                     "Source":status.source}
            empty_data = empty_data.append(tweet_elem, ignore_index = True)
    empty_data.to_csv("OLAprilMay19.csv", mode="a")

for i in range(chunks):
        batch = ids[i*50:(i+1)*50]
        result = fetch_tw(batch)